### Fit ffs vs T for R10_F2_5p13

In [ ]:
import corner
from pathlib import Path
from lmfit import Parameters
import numpy as np

from rrfit.ffs import fitIterated_FFS_gammaFixed, plot_ffs_vs_temp, emcee_ffs

from betata import plt
from betata.resonator_studies.trace import load_fitted_traces, plot_fitted_trace
from betata.resonator_studies.resonator import load_resonator

CWD = Path.cwd()

Specify resonator file

In [ ]:
resonator_name = "R10_F2_5p13"
resonator_file = CWD / f"out/resonator_studies/{resonator_name}.h5"

Load resonator data

In [ ]:
resonator = load_resonator(resonator_file)
resonator.traces = load_fitted_traces(resonator_file)

Plot ffs vs T

In [ ]:
plot_ffs_vs_temp([resonator])

Inspect specified S21 traces and exclude additional traces if needed

In [ ]:
powers = []
min_temp, max_temp = np.inf, -np.inf
for trace in resonator.traces:
    if min_temp < trace.temperature < max_temp and not trace.is_excluded: 
        plot_fitted_trace(trace, resonator_name)

In [ ]:
traces_to_exclude = []
traces_to_include = []

for trace in resonator.traces:

    date = int(trace.filename.split("_")[0].split("-")[2])
    if date < 10: # reject traces from first measurement period, before thermal cycle
        trace.is_excluded = True

    if np.isnan(trace.temperature):
        trace.is_excluded = True
    if trace.id in traces_to_exclude:
        trace.is_excluded = True
    if trace.id in traces_to_include:
        trace.is_excluded = False

Perform initial iterated fit

In [ ]:
print(f"alpha: {resonator.alpha_bare}")
print(f"fr_ref: {resonator.traces[0].fr}")

In [ ]:
# fit parameters
init_params = Parameters()
init_params.add("Q_TLS0", value=5e5, min=0)
init_params.add("alpha", value=resonator.alpha_bare, min=0, max=1)
init_params.add("tc", value=0.45, min=0.2, max=0.8)
init_params.add("f0", value=resonator.traces[0].fr, min=3e9)

boundsDict_FFS = {
    "Q_TLS0": (1e5, 1e6),
    "alpha":  (0.8, 1.0),
    "tc":     (0.3, 0.6),
    "f0": (5e9, 5.5e9),
}

initDict_best, finalDict_best, red_chi2_best, figList_best = fitIterated_FFS_gammaFixed(
    device=resonator,
    boundsDict=boundsDict_FFS,
    init_params=init_params,
    numIter=200,
    makePlot=True,
    powers=None,
    fit_QTLS0=True,
    fit_QP=True,
    fit_f0=True,
)
chi2Fig, countFig, probFig, initFig, fittedFig = figList_best

In [ ]:
resonator.ffs_fit_params

MCMC

In [ ]:
emcee_kws = {
    "nwalkers": 250,
    "burn": 500,
    "steps": 10000,
    "thin": 25,
    "is_weighted": False,
    "lnsigma_params": (np.log(0.1), -10, 10)
}
emcee_result = emcee_ffs(resonator, sample_only_QP=False, **emcee_kws)

In [ ]:
corner_fig = plt.figure(figsize=(8, 8))
selected_params = ["alpha", "tc"]
selected_param_slc = slice(1, 3)
emcee_plot = corner.corner(
    emcee_result.flatchain[selected_params],
    labels=selected_params,
    truths=list(emcee_result.params.valuesdict().values())[selected_param_slc],
    fig=corner_fig,
)
emcee_plot.tight_layout()